In [122]:
import pandas as pd
import numpy as np
import opendatasets as od
import time
import math
import datetime
from torchsummary import summary
import torch.nn as nn
pd.options.mode.chained_assignment = None

import torch
from sklearn.model_selection import train_test_split

In [123]:
dataset_url = 'https://www.kaggle.com/competitions/new-york-city-taxi-fare-prediction'
od.download(dataset_url)

Skipping, found downloaded files in "./new-york-city-taxi-fare-prediction" (use force=True to force download)


In [124]:
df = pd.read_csv('./NYCTaxiFares_small.csv')
df['pickup_datetime'].max()

'2010-04-25 03:59:42 UTC'

In [125]:
## Running this cell is really fucking slow, but it works.
df['pickup_datetime'] = pd.to_datetime(df['pickup_datetime'])

def havardsine_distance(lat1, long1, lat2, long2):
    r = 6371

    phi1 = np.radians(lat1)
    phi2 = np.radians(lat2)

    delta_phi = np.radians(lat2 - lat1)
    delta_lambda = np.radians(long2 - long1)
    a = np.sin(delta_phi / 2.0) ** 2 + np.cos(phi1) * np.cos(phi2) * np.sin(delta_lambda / 2.0) ** 2
    c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1 - a))
    d = (r * c) # convert to kilometer
    return d

df['dist_km'] = df.apply(lambda row: havardsine_distance(row.pickup_latitude, row.pickup_longitude, row.dropoff_latitude, row.dropoff_longitude), axis=1)

airport_cords = [
    [40.691547, -74.180202], # NEWARK
    [40.773281, -73.869845], # LAGUARDIA
]

# When I wrote this code, only God and I knew what was going on. Today, only God knows.
def close_to_airport(values):
    pickup_latitude = values['pickup_latitude']
    pickup_longitude = values['pickup_longitude']
    dropoff_latitude = values['dropoff_latitude']
    dropoff_longitude = values['dropoff_longitude']

    # JFK is special, it needs a bigger radius
    jfkpickup = havardsine_distance(pickup_latitude, pickup_longitude, 40.645042, -73.786928) <= 1.00
    jfkdropoff = havardsine_distance(dropoff_latitude, dropoff_longitude, 40.645042, -73.786928) <= 1.00

    # Newark is a small airport but their parking log is BIG.
    newarkpickup = havardsine_distance(pickup_latitude, pickup_longitude, 40.691547, -74.180202) <= 1.00
    newarkdropoff = havardsine_distance(dropoff_latitude, dropoff_longitude, 40.691547, -74.180202) <= 1.00

    # Third largest, it's an abstract airport with a stretched out U shaped parking place
    # LaGuardia
    laguardiapickup = havardsine_distance(pickup_latitude, pickup_longitude, 40.773855, -73.871712) <= 0.50
    laguardiadropoff = havardsine_distance(dropoff_latitude, dropoff_longitude, 40.773855, -73.871712) <= 0.50

    return 1 if jfkpickup or jfkdropoff or newarkpickup or newarkdropoff or laguardiapickup or laguardiadropoff else 0

df['is_by_airport'] = df.apply(close_to_airport, axis=1)

In [126]:
my_time = df['pickup_datetime'][0]

In [127]:
df['EDTdate'] = df['pickup_datetime'] - pd.Timedelta(hours=4)
df['Hour'] = df['EDTdate'].dt.hour
df['AMorPM'] = np.where(df['Hour'] < 12, 0, 1)

In [128]:
df['Weekday'] = pd.to_numeric(df['EDTdate'].dt.strftime("%w")) # an int between 0 - 6, representing the weekdays
df['Month'] = pd.to_numeric(df['EDTdate'].dt.strftime("%m")) # an int between 0 - 6, representing the weekdays

#Formatting this correctly, we can use the dates to get the historical weather data for the dates
df['FullDate'] = df['EDTdate'].dt.strftime('%x')
#print(df['FullDate'].min())
#print(df['FullDate'].max())
print(df['FullDate'].unique())

['04/19/10' '04/17/10' '04/11/10' '04/16/10' '04/22/10' '04/23/10'
 '04/15/10' '04/20/10' '04/21/10' '04/14/10' '04/13/10' '04/12/10'
 '04/24/10' '04/18/10']


In [129]:
'''
api_key_openweather = "661b8c5b6e790b9cf55403d0af1cebe8"
new_york_middle_lat = ""
new_york_middle_long = ""
unique = np.sort(df['FullDate'].unique())
start = time.mktime(datetime.datetime.strptime(unique[0], "%m/%d/%y").timetuple())
end = time.mktime(datetime.datetime.strptime(unique[len(unique) - 1], "%m/%d/%y").timetuple())
url = f"https://history.openweathermap.org/data/2.5/history/city?lat={new_york_middle_lat}&lon={new_york_middle_long}&type=hour&start={start}&end={end}&appid={api_key_openweather}"

print(url)
'''
# Historical weather data, not working

'\napi_key_openweather = "661b8c5b6e790b9cf55403d0af1cebe8"\nnew_york_middle_lat = ""\nnew_york_middle_long = ""\nunique = np.sort(df[\'FullDate\'].unique())\nstart = time.mktime(datetime.datetime.strptime(unique[0], "%m/%d/%y").timetuple())\nend = time.mktime(datetime.datetime.strptime(unique[len(unique) - 1], "%m/%d/%y").timetuple())\nurl = f"https://history.openweathermap.org/data/2.5/history/city?lat={new_york_middle_lat}&lon={new_york_middle_long}&type=hour&start={start}&end={end}&appid={api_key_openweather}"\n\nprint(url)\n'

In [130]:
# lets have some weather-y fuuuuuuuuuuuun-fuuuuuun-function
# see this for doc https://www1.ncdc.noaa.gov/pub/data/cdo/documentation/GHCND_documentation.pdf
weather_data = pd.read_csv('nyc-weather-data.csv')
weather_labels = ['SNWD', 'SNOW', 'AWND'] #snow depth, did it snow
for index, item in enumerate(weather_labels):
    weather_data.loc[weather_data[item] <= -9999, item] = 0 # -9999 means that there is no data

# df['dist_km'] = df.apply(lambda row: havardsine_distance(row.pickup_latitude, row.pickup_longitude, row.dropoff_latitude, row.dropoff_longitude), axis=1)

weather_data['DATE'] = weather_data.apply(lambda row: datetime.datetime.strptime(str(row.DATE), '%Y%m%d').strftime('%m/%d/%y'), axis=1)
weather_data = weather_data[['DATE','PRCP','SNWD','SNOW','TMAX','TMIN','AWND']]

weather_data['TMIN'] = (weather_data['TMIN'] / 10) # the API returns degrees in an old format. We therefore convert the value to a tenth of its own form. Check the docu for this.
weather_data['TMAX'] = (weather_data['TMAX'] / 10)
weather_data['PRCP'] = weather_data['PRCP'] / 10

weather_data['AWND'] = weather_data['AWND'] / 2.237 # meters per second

df.rename(columns={'FullDate':'DATE'}, inplace=True)

merged_df = pd.merge(df, weather_data, how="left",on="DATE")

In [164]:
network_data = merged_df[['pickup_latitude', 'pickup_longitude', 'dropoff_latitude', 'dropoff_longitude', 'dist_km', 'passenger_count', 'is_by_airport', 'Hour', 'AMorPM', 'Weekday', 'PRCP', 'TMAX', 'TMIN', 'AWND', 'fare_amount']]

network_data['pickup_longitude'] = network_data.apply(lambda row: abs(row.pickup_longitude), axis=1)

network_data['dropoff_longitude'] = network_data.apply(lambda row: abs(row.pickup_longitude), axis=1)

network_data.iloc[:,0:-1] = network_data.iloc[:,0:-1].apply(lambda x: (x-x.mean())/ x.std(), axis=0)

prediction = network_data['fare_amount']

network_data.drop('fare_amount', axis=1, inplace=True)
X = network_data.values

y = prediction.values

(120000,)


In [165]:
train_X, test_X, train_y, test_y = train_test_split(X, y, test_size=0.1)

In [194]:
X = torch.tensor(train_X, dtype=torch.float32)
y = torch.tensor(train_y, dtype=torch.long)
test_X_tensor = torch.Tensor(test_X)
test_y_tensor = torch.LongTensor(test_y)

train_ds = torch.utils.data.TensorDataset(X, y)
test_ds = torch.utils.data.TensorDataset(test_X_tensor, test_y_tensor)

train_dl = torch.utils.data.DataLoader(train_ds, batch_size=128, shuffle=True)
test_dl = torch.utils.data.DataLoader(test_ds, batch_size=128, shuffle=False)

In [195]:
class NYCTaxiPredictor(nn.Module):
    def __init__(self):
        super(NYCTaxiPredictor, self).__init__()
        self.layers = nn.Sequential(
            nn.Linear(14, 64),
            nn.ReLU(),
            nn.Linear(64, 64),
            nn.BatchNorm1d(64),
            nn.Dropout(0.5),
            nn.ReLU(),
            nn.Linear(64, 64),
            nn.BatchNorm1d(64),
            nn.Dropout(0.5),
            nn.ReLU(),
            nn.Linear(64, 128),
            nn.LogSoftmax(dim=-1),
        )

    def forward(self, x):
        return self.layers(x)


model = NYCTaxiPredictor()
summary(model, tuple([14]))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Linear-1                   [-1, 64]             960
              ReLU-2                   [-1, 64]               0
            Linear-3                   [-1, 64]           4,160
       BatchNorm1d-4                   [-1, 64]             128
           Dropout-5                   [-1, 64]               0
              ReLU-6                   [-1, 64]               0
            Linear-7                   [-1, 64]           4,160
       BatchNorm1d-8                   [-1, 64]             128
           Dropout-9                   [-1, 64]               0
             ReLU-10                   [-1, 64]               0
           Linear-11                  [-1, 128]           8,320
       LogSoftmax-12                  [-1, 128]               0
Total params: 17,856
Trainable params: 17,856
Non-trainable params: 0
---------------------------------

In [200]:
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=0.0001)
criterion = nn.CrossEntropyLoss()
loss_values = []
for epochs in range(100):
    for data in train_dl:
        X, y = data
        optimizer.zero_grad()
        output = model.forward(X)
        #print(output.shape, y.shape)
        loss = nn.functional.nll_loss(output, y)
        #loss_values.append(loss.item())
        #print(loss.item())
        loss.backward()
        optimizer.step()
    print(loss.data)

tensor(2.2268)
tensor(1.9952)
tensor(1.8045)
tensor(1.9620)
tensor(1.7107)
tensor(1.9045)
tensor(1.9211)
tensor(2.1903)
tensor(1.8914)
tensor(1.9807)
tensor(2.0798)
tensor(2.0558)
tensor(1.9794)
tensor(1.9588)
tensor(1.9343)
tensor(1.8731)
tensor(1.8536)
tensor(1.9154)
tensor(1.9519)
tensor(2.1043)
tensor(2.1025)
tensor(1.9585)
tensor(1.8975)
tensor(2.2773)
tensor(2.0163)
tensor(2.0783)
tensor(2.1936)
tensor(1.9755)
tensor(2.1947)
tensor(1.8795)
tensor(2.2489)
tensor(2.0573)
tensor(1.9483)
tensor(2.1992)
tensor(1.7760)
tensor(1.8176)
tensor(1.9772)
tensor(1.9144)
tensor(2.0809)
tensor(2.2703)
tensor(1.6853)
tensor(1.7152)
tensor(1.9469)
tensor(1.8466)
tensor(1.9779)
tensor(2.1600)
tensor(2.0582)
tensor(2.1840)
tensor(1.9981)
tensor(2.1295)
tensor(1.9081)
tensor(2.0169)
tensor(1.7626)
tensor(1.9845)
tensor(2.0790)
tensor(1.8449)
tensor(2.0141)
tensor(2.1006)
tensor(1.9803)
tensor(1.9601)
tensor(1.8768)
tensor(1.8915)
tensor(1.8902)
tensor(2.0201)
tensor(1.8544)
tensor(1.9564)
tensor(1.9

In [201]:
total = 0
correct = 0
with torch.no_grad(): # no need to keep track of the gradient. The backward() is doing it
  for data in test_dl:
    X, y = data
    output = model(X)
    for idx, val in enumerate(output):
      print(torch.argmax(val), y[idx])
      if torch.argmax(val) == y[idx]:
        correct += 1
      total += 1
  print("Accuracy:", round(correct/total, 3) * 100)

tensor(6) tensor(6)
tensor(14) tensor(13)
tensor(8) tensor(6)
tensor(14) tensor(17)
tensor(6) tensor(6)
tensor(15) tensor(14)
tensor(6) tensor(4)
tensor(10) tensor(9)
tensor(8) tensor(9)
tensor(4) tensor(4)
tensor(12) tensor(14)
tensor(8) tensor(8)
tensor(8) tensor(8)
tensor(8) tensor(6)
tensor(6) tensor(8)
tensor(8) tensor(8)
tensor(8) tensor(10)
tensor(16) tensor(13)
tensor(4) tensor(4)
tensor(8) tensor(8)
tensor(8) tensor(6)
tensor(8) tensor(8)
tensor(10) tensor(9)
tensor(4) tensor(4)
tensor(10) tensor(9)
tensor(8) tensor(8)
tensor(3) tensor(4)
tensor(8) tensor(9)
tensor(6) tensor(6)
tensor(16) tensor(15)
tensor(12) tensor(10)
tensor(8) tensor(10)
tensor(4) tensor(6)
tensor(8) tensor(12)
tensor(4) tensor(5)
tensor(3) tensor(3)
tensor(10) tensor(9)
tensor(4) tensor(4)
tensor(4) tensor(4)
tensor(6) tensor(8)
tensor(4) tensor(6)
tensor(7) tensor(16)
tensor(8) tensor(8)
tensor(8) tensor(12)
tensor(12) tensor(12)
tensor(8) tensor(9)
tensor(15) tensor(14)
tensor(6) tensor(8)
tensor(4) ten